In [1]:
# Dependencies
import tweepy
import json
import numpy as np
from config import consumer_key, consumer_secret, access_token, access_token_secret
from datetime import datetime
import pandas as pd

In [2]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret

In [4]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [5]:
# Target Search Term
target_term = "GOOGL"

In [12]:
# Lists to hold sentiments
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
tweet_text = []
tweet_times = []
tweet_id = []
time_diffs = []
tweet_user = []
tweet_handle = []
tweet_followers = []
tweet_language = []

In [13]:
# Grab 25 tweets
public_tweets = api.search(target_term, count=100, result_type="recent")

print(json.dumps(public_tweets,indent = 4))

{
    "statuses": [
        {
            "created_at": "Sun Jun 24 15:30:14 +0000 2018",
            "id": 1010907961676500993,
            "id_str": "1010907961676500993",
            "text": "ACTIVE TRADERS Try one of these FREE trading guides:\n\nhttps://t.co/2jSM4Fn7z1\n\n$NFLX $TSLA $AAPL $WMT $GS $FB $AMZN $GOOGL $NVDA",
            "truncated": false,
            "entities": {
                "hashtags": [],
                "symbols": [
                    {
                        "text": "NFLX",
                        "indices": [
                            79,
                            84
                        ]
                    },
                    {
                        "text": "TSLA",
                        "indices": [
                            85,
                            90
                        ]
                    },
                    {
                        "text": "AAPL",
                        "indices": [
                            91

In [14]:
# Loop through all tweets
for tweet in public_tweets["statuses"]:

    # Run Vader Analysis on each tweet
    compound = analyzer.polarity_scores(tweet["text"])["compound"]
    pos = analyzer.polarity_scores(tweet["text"])["pos"]
    neu = analyzer.polarity_scores(tweet["text"])["neu"]
    neg = analyzer.polarity_scores(tweet["text"])["neg"]
    
    raw_time = tweet["created_at"]
    converted_time = datetime.strptime(raw_time, "%a %b %d %H:%M:%S %z %Y")
    tweet_times.append(converted_time)
    
       
    text = tweet["text"]
    tweet_text.append(text)
    
    tw_id = tweet["id"]
    tweet_id.append(tw_id)
    
    user = tweet["user"]["name"]
    tweet_user.append(user)
    
    handle = tweet["user"]["screen_name"]
    tweet_handle.append(handle)
    
    followers = tweet["user"]["followers_count"]
    tweet_followers.append(followers)
    
    language =  tweet["user"]["lang"]
    tweet_language.append(language)
    
        
    # Add each value to the appropriate array
    compound_list.append(compound)
    positive_list.append(pos)
    negative_list.append(neg)
    neutral_list.append(neu)
    
    
   

In [15]:
converted_length = len(tweet_times)

time_diffs = []

for x in range(converted_length - 1):
    time_diff = tweet_times[x] - tweet_times[x + 1]
#     print(f'time diff: {time_diff}')
#     print(f'time diff (in seconds): {time_diff.seconds}')
#     print(f'time diff (in minutes): {time_diff.seconds / 60}')
#     print(f'time diff (in hours): {time_diff.seconds / 3600}')

    # convert time_diff to hours
    time_diff = time_diff.seconds
    time_diffs.append(time_diff)

time_diffs.append(0)
print(len(time_diffs))
print(len(tweet_times))
print(len(tweet_text))
print(len(compound_list))

97
97
97
97


In [16]:
tweet_data = pd.DataFrame({'Name':tweet_user,
                           'Handle':tweet_handle,
                           'Followers':tweet_followers,
                           'Language' : tweet_language,
                            'Id':tweet_id,
                            'Text': tweet_text,
                           'Time Stamp':tweet_times,
                           'Time Delta (seconds)':time_diffs,
                           'Compound' : compound_list,
                           'positive': positive_list,
                           'negative': negative_list,
                           'neutral': neutral_list})
   
for index, row in tweet_data.iterrows():

    tweet_data["Date"] =  pd.to_datetime(row["Time Stamp"],unit='s').date()
    
tweet_data.head(25)

,Compound,Followers,Handle,Id,Language,Name,Text,Time Delta (seconds),Time Stamp,negative,neutral,positive,Date
0,0.8159,298,ExactOptionPick,1010907961676500993,en,Exact Option Picks,ACTIVE TRADERS Try one of these FREE trading g...,14,2018-06-24 15:30:14+00:00,0.0,0.695,0.305,2018-06-24
1,0.3818,148,Plantandwatch,1010907901530116096,en,Bringithome,This will pour gas on #smartglass growth. $GOO...,13,2018-06-24 15:30:00+00:00,0.0,0.843,0.157,2018-06-24
2,0.4005,33,praveen_JR,1010907846467309569,en,Praveen J R,RT @Sandro_power: $GOOGL $GOOG more gains for...,67,2018-06-24 15:29:47+00:00,0.0,0.870,0.130,2018-06-24
3,0.0000,2003,seneco1907,1010907568796028929,tr,seneco,@vatanseven0007 fox tv canlı yayın yaz googl'a,170,2018-06-24 15:28:40+00:00,0.0,1.000,0.000,2018-06-24
4,0.0000,407,XBjansan,1010906854094880768,en,Jansan 👽,Putang inang net shop to. Can't access Gdrive ...,149,2018-06-24 15:25:50+00:00,0.0,1.000,0.000,2018-06-24
5,0.6249,2590,sfef84,1010906228443287553,en,Steven Feffer,Woukd be awesome if $GOOGL does Pixel XL/watc...,41,2018-06-24 15:23:21+00:00,0.0,0.823,0.177,2018-06-24
6,0.4215,68,brainhoits,1010906055860260865,en,harry anderson,$NFLX $TWTR = my favorites. $NVDA $BABA $...,65,2018-06-24 15:22:40+00:00,0.0,0.811,0.189,2018-06-24
7,0.0000,483,newamericano,1010905786623635457,en,Luis Gutierrez,$GOOGL Alphabet reports in one month. Volatili...,14,2018-06-24 15:21:35+00:00,0.0,1.000,0.000,2018-06-24
8,0.0000,7283,bgamall,1010905725487308801,en,Boycott the Lenders,The Waymo Tesla Self Driving Con $GOOGL $TSLA...,170,2018-06-24 15:21:21+00:00,0.0,1.000,0.000,2018-06-24
9,0.0000,68,brainhoits,1010905012724215808,en,harry anderson,"$TWTR instead of an options exp sell down , w...",204,2018-06-24 15:18:31+00:00,0.0,1.000,0.000,2018-06-24


In [18]:
tweet_data.to_csv("tweet_data.csv")



In [ ]:
# Store the Average Sentiments
sentiment = {"Compound": np.mean(compound_list),
             "Positive": np.mean(positive_list),
             "Neutral": np.mean(neutral_list),
             "Negative": np.mean(negative_list)}



In [ ]:
# Print the Sentiments
print(sentiment)
print("")